In [1]:
import pickle
import os
import numpy as np
import torch
import random
import soundfile as sf
from factory.AutoVC import AutoVC

In [2]:
from util.evaluate import Evaluator

In [3]:
NAME = "autovc2"
ROOT = "train_data_vctk_ver2"
TAGET = f"mos_data_{NAME}"
MODEL = f"{NAME}_en.pt"

In [4]:
class Config:
    def __init__(self,root):
        self.root = root
        self.num_speaker = 40
        self.batch_size = 2
        self.erroment_num = 16
        self.max_uttr_idx = 60
        self.len_crop = 176
        self.device = "cuda:0"
        self.embedder = None
        self.pick_speaker = sorted(next(iter(os.walk(root)))[1][:self.num_speaker])
        self.metadata = pickle.load(open(f'{root}/train.pkl', "rb"))

In [5]:
config = Config(ROOT)

In [6]:
E = Evaluator(config)

In [7]:
isadain = False

In [11]:
for speaker in config.pick_speaker:
    try:
        os.mkdir(f"{TAGET}/{speaker}")
    except:
        pass

In [12]:
org = AutoVC(44,256,512,22).to(config.device)
org.load_state_dict(torch.load(f"model/{MODEL}", map_location=config.device))

<All keys matched successfully>

In [13]:
for source_id, data in enumerate(E.metadata):
    sp_s = data[0]
    print(f"Processing {sp_s}")
    for target_id,_ in enumerate(E.metadata): 
        for i in range(5):
            _,_, mel_trans_org = E.get_trans_mel(org, source_id, target_id,i+2,isadain,isPlay=True)
            wav_org = E.get_wavs(mel_trans_org.transpose(2,1)).detach().cpu().numpy()[0].astype(np.float32)
            sf.write(f"{TAGET}/{sp_s}/{source_id}_{target_id}_{i}.wav",wav_org,samplerate=22050)

Processing p225
Processing p226
Processing p227
Processing p228
Processing p229
Processing p230
Processing p231
Processing p232
Processing p233
Processing p234
Processing p236
Processing p237
Processing p238
Processing p239
Processing p240
Processing p241
Processing p243
Processing p244
Processing p245
Processing p246
Processing p247
Processing p248
Processing p249
Processing p250
Processing p251
Processing p252
Processing p253
Processing p254
Processing p255
Processing p256
Processing p257
Processing p258
Processing p259
Processing p260
Processing p261
Processing p262
Processing p263
Processing p264
Processing p265
Processing p266
